In [1]:
import pandas as pd
import numpy as np

In [90]:
# load
loan_result=pd.read_csv("../data/loan_result.csv")
log_data=pd.read_csv("../data/log_data.csv")
user_spec=pd.read_csv("../data/user_spec.csv")

# columns localize
loan_result.columns=['신청서_번호', '한도조회_일시', '금융사_번호', '상품_번호', '승인한도', '승인금리', '신청_여부(y)']
log_data.columns=['유저_번호', '행동명', '행동일시', '운영체제', '앱_버전', '일_코드']
user_spec.columns=['신청서_번호', '유저_번호', '유저_생년월일', '유저_성별', '생성일시', '한도조회_당시_유저_신용점수', '연소득', '근로형태', '입사연월', '고용형태', '주거소유형태', '대출희망금액', '대출_목적', '개인회생자_여부', '개인회생자_납입_완료_여부', '기대출수', '기대출금액'] # 건강보험 타입이 정의서에는 있는데 데이터엔 없는듯

In [91]:
log_data.shape

(17843993, 6)

In [92]:
# log_data sort by time(asc)
log_data=log_data.sort_values('행동일시').reset_index(drop=True)
log_data.head()

,유저_번호,행동명,행동일시,운영체제,앱_버전,일_코드
0,125129,UseLoanManage,2022-03-01 00:00:00,Android,3.8.0,2022-03-01
1,385811,Login,2022-03-01 00:00:00,Android,3.8.0,2022-03-01
2,720911,OpenApp,2022-03-01 00:00:00,iOS,3.6.0,2022-03-01
3,706858,UseLoanManage,2022-03-01 00:00:03,Android,3.8.0,2022-03-01
4,512341,EndLoanApply,2022-03-01 00:00:03,Android,3.8.0,2022-03-01


In [93]:
log_data['행동일시'] = pd.to_datetime(log_data['행동일시'])
log_data['일_코드'] = pd.to_datetime(log_data['일_코드'])
loan_result['한도조회_일시'] = pd.to_datetime(loan_result['한도조회_일시'])
user_spec['생성일시'] = pd.to_datetime(user_spec['생성일시'])

In [119]:
users=set(user_spec['유저_번호'])
log_user=set(log_data['유저_번호'])
count=[]

for num in log_user:
    if num in users:
        count.append(num)

print(len(count))

370436


In [ ]:
from tqdm import tqdm

user_df=user_spec.copy()
log_df=log_data.copy()

# 각 13시간 정도...

for i in tqdm(range(len(user_spec))):
    if user_spec['유저_번호'].iloc[i] not in count:
        user_df.drop([i])



## 주요 경로
### 1. 경로의 종류
#### 1) 5개의 스크린
- 한도조회시작(StartLoanApply), 대출관리서비스 이용(UseLoanManage), 여윳돈 계산기 이용(UsePrepayCalc), DSR 계산기 이용(UseDSRCalc), KCB 신용정보 조회(GetCreditInfo)
- ['StartLoanApply', 'UseLoanManage', 'UsePrepayCalc', 'UseDSRCalc', 'GetCreditInfo']
#### 2) 시작점-종료점
- OpenApp부터 그 다음 OpenApp 까지

### 2. 슬라이싱
- 0. log_data에서 5개의 스크린만 뽑기
- 1. log_data를 user별로 
- 2. OpenApp 앞뒤를 쪼개서 여러개의 리스트로 합친
- 3. 전체 route_list에 넣음
- 4. 겹치는 것의 종류 확인
    + 5!+4!+3!+2!+1!=153 보다 많다면 쪼개야하나.. 일단 최대 길이 찾아서 대기..
- 5. value_counts()

In [165]:
screens=['StartLoanApply', 'UseLoanManage', 'UsePrepayCalc', 'UseDSRCalc', 'GetCreditInfo']
main_route_log_df=log_data[(log_data['행동명']=="OpenApp")|
                           (log_data['행동명']==screens[0])|
                           (log_data['행동명']==screens[1])|
                           (log_data['행동명']==screens[2])|
                           (log_data['행동명']==screens[3])|
                           (log_data['행동명']==screens[4])
                           ]
main_route_log_df.reset_index(drop=True, inplace=True)
len(main_route_log_df), len(log_data), len(set(main_route_log_df['유저_번호']))

(9587604, 17843993, 574778)

In [ ]:
users=list(set(main_route_log_df['유저_번호']))
route_list=[]

# target의 idx들(복수)를 list로 추출
def list_idx(arr, target):
    return [i for i in range(len(arr)) if arr[i]==target]

# 유저별로 OpenApp 기준 슬라이싱
for i in tqdm(range(len(users))):
    df=main_route_log_df[main_route_log_df['유저_번호']==users[i]]
    idx_list=list_idx(list(df['행동명']), 'OpenApp')
    if len(idx_list)>0 and len(df)>0:
        for i in range(len(idx_list)):
            start=idx_list[i]
            try:
                end=idx_list[i+1]
            except:
                end=len(df)
            route_list.append(list(df['행동명'].iloc[start:end]))
        # route_list.append([df['행동명'].iloc[idx_list[i]:min(idx_list[i+1], len(df))] for i in range(len(idx_list))])
    else: continue

In [179]:
len(route_list)

3460762

In [192]:
route_set = []

for i in tqdm(range(len(route_list))):
    if route_list[i] not in route_set:
        route_set.append(route_list[i])

route_set.sort()
len(route_set)

100%|██████████| 3460762/3460762 [00:13<00:00, 260638.53it/s]


24883

In [185]:
np.save('../data/route_set', np.array(route_set)) # 중복 제거한 루트 종류

In [398]:
from asyncore import poll3
from itertools import permutations
from itertools import combinations

log_init=[('StartLoanApply'), ('UseLoanManage'), ('UsePrepayCalc'), ('UseDSRCalc'), ('GetCreditInfo')]
log_columns=[('StartLoanApply'), ('UseLoanManage'), ('UsePrepayCalc'), ('UseDSRCalc'), ('GetCreditInfo')]

p2=list(permutations(log_init, 2))
p3=list(permutations(log_init, 3))
p4=list(permutations(log_init, 4))
p5=list(permutations(log_init, 5))

n1=5
n2=len(p2)
n3=len(p3)
n4=len(p4)
n5=len(p5)

for arr in p2:
    log_columns.append(arr)
for arr in p3:
    log_columns.append(arr)
for arr in p4:
    log_columns.append(arr)
for arr in p5:
    log_columns.append(arr)
    
log_columns.append((''))

len(log_columns), n1+n2+n3+n4+n5+1 # 5개의 스크린의 1, 2, 3, 4, 5 순열 합

(326, 326)

In [399]:
log_ngram_df=pd.DataFrame([0]*len(log_columns), index=log_columns, columns=['count'])

count=0
for i in range(len(route_list)):
    if len(route_list[i])==1: count+=1
log_ngram_df.loc['']=count

log_ngram_df

,count
StartLoanApply,0
UseLoanManage,0
UsePrepayCalc,0
UseDSRCalc,0
GetCreditInfo,0
...,...
"(GetCreditInfo, UseDSRCalc, UseLoanManage, StartLoanApply, UsePrepayCalc)",0
"(GetCreditInfo, UseDSRCalc, UseLoanManage, UsePrepayCalc, StartLoanApply)",0
"(GetCreditInfo, UseDSRCalc, UsePrepayCalc, StartLoanApply, UseLoanManage)",0
"(GetCreditInfo, UseDSRCalc, UsePrepayCalc, UseLoanManage, StartLoanApply)",0


In [401]:
# screens=['StartLoanApply', 'UseLoanManage', 'UsePrepayCalc', 'UseDSRCalc', 'GetCreditInfo']

for i in tqdm(range(len(route_list))):
    if len(route_list[i])==1: continue
    for n in range(1, len(route_list[i])):
        # 원소 안의 2~5개의 순서 지킨 조합에서
            # range에서도 1 빠지고 슬라이싱에서도 1 빠져서 머리 터지는 줄
        for k in range(n+1, min(n+1+5, len(route_list[i])+1)):
            # print(tuple(route_list[i][n:k]))
            # 하나일 때랑 여러개 일 때 들어가는 [] 계층이 다름
            if len(route_list[i][n:k])==1: 
                log_ngram_df.loc[route_list[i][n:k]]+=1
            else: 
                try: 
                    log_ngram_df.loc[[tuple(route_list[i][n:k])]]+=1
                except KeyError:
                    continue
log_ngram_df.head()

100%|██████████| 3460762/3460762 [1:05:21<00:00, 882.49it/s] 


,count
StartLoanApply,1694042
UseLoanManage,1423937
UsePrepayCalc,5784
UseDSRCalc,3660
GetCreditInfo,2113765


In [404]:
log_ngram_df.head(20)

,count
StartLoanApply,1694042
UseLoanManage,1423937
UsePrepayCalc,5784
UseDSRCalc,3660
GetCreditInfo,2113765
"(StartLoanApply, UseLoanManage)",138206
"(StartLoanApply, UsePrepayCalc)",6
"(StartLoanApply, UseDSRCalc)",6
"(StartLoanApply, GetCreditInfo)",175437
"(UseLoanManage, StartLoanApply)",127845


In [402]:
log_ngram_df.to_csv("../data/log_ngram_df.csv", index=False)